<img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/horizontal-primary-light.png" alt="he-black-box" width="600"/>


# Homomorphic Encryption using Duet: Data Owner
## Tutorial 0: Basic operations


Welcome!
This tutorial will show you how to use Duet with homomorphic encryption and some use cases. This notebook illustrates the Data Owner view on the operations.

We will focus on Duet's integration with [TenSEAL](https://github.com/OpenMined/TenSEAL). 
TenSEAL is a Python library for doing homomorphic encryption operations on tensors. It's built on top of [Microsoft SEAL](https://github.com/Microsoft/SEAL), a C++ library implementing the BFV and CKKS homomorphic encryption schemes.


If you want to learn more about TenSEAL, we recommend the following tutorials:
- ['Tutorial 0 - Getting Started'](https://github.com/OpenMined/TenSEAL/blob/master/tutorials/Tutorial%200%20-%20Getting%20Started.ipynb).
- ['Tutorial 1: Training and Evaluation of Logistic Regression on Encrypted Data'](https://github.com/OpenMined/TenSEAL/blob/master/tutorials/Tutorial%201%20-%20Training%20and%20Evaluation%20of%20Logistic%20Regression%20on%20Encrypted%20Data.ipynb).
- ['Tutorial 2: Working with Approximate Numbers'](https://github.com/OpenMined/TenSEAL/blob/master/tutorials/Tutorial%202%20-%20Working%20with%20Approximate%20Numbers.ipynb).


Let's now start the tutorial with a brief review of what homomorphic encryption is, but keep in mind that you don't need to be a crypto expert to use these features.

## Homomorphic Encryption

__Definition__ : Homomorphic encryption (HE) is a technique that allows computations to be made on ciphertexts and generates results that when decrypted, corresponds to the result of the same computations made on plaintexts.

<img src="https://github.com/OpenMined/TenSEAL/raw/master/tutorials/assets/he-black-box.png" alt="he-black-box" width="600"/>

This means that an HE scheme lets you encrypt two numbers *X* and *Y*, add their encrypted versions so that it gets decrypted to *X + Y*, the addition could have been a multiplication as well. 

### Setup

All modules are imported here, make sure everything is installed by running the cell below.

In [ ]:
import syft as sy
import tenseal as ts
import pytest

sy.load("tenseal")

### Start Duet Data Owner instance

In [ ]:
# Start Duet local instance
duet = sy.launch_duet(loopback=True)

### Theory: Homomorphic encryption schemes

__TenSEAL__ supports two encryption schemes:
 - __BFV__, a scheme for operations on integers.
 - __CKKS__, a scheme for operations on approximate numbers. This scheme is much better suited for ML applications and we will focus more on it.
 
There are a few major steps for each scheme:
 1. __Keys Generation__: in this step, we generate public and private keys that will be used for encryption/decryption.
 2. __Encryption__: this is the process of converting a plaintext into an encrypted ciphertext. This step requires an encryption key(or a public key).
 3. __Decryption__: this is the process of converting a ciphertext back into a plaintext. This step requires a decryption key(or a secret key). This step cannot be done on the Data Scientist endpoint.

### Theory: Homomorphic encryption parameters

__TenSEAL__ requires a few parameters to set the keys up:
 - __The polynomial modulus degree(poly_modulus_degree).__ This parameter directly affects the size of the ciphertext, the security of the scheme(bigger is better), but also the computational performance of the scheme(bigger is worse)
 - __The coefficient modulus sizes(coeff_mod_bit_sizes).__ This parameter is an array of bit sizes and directly affects the size of the ciphertext, the security of the scheme(bigger is worse), and the depth of computation allowed in the encrypted space(longer is better).
 - __The scaling factor(global_scale).__ This parameter is only used for the approximate schemes(CKKS) and directly affects the precision of computation and decryption.

### Theory: Homomorphic encryption keys

__TenSEAL__ generates a few keys internally, each with another use case:
 - __The Private Key(or the secret/decryption key)__. This key is used for decrypting ciphertexts, and it is used to derive the other keys. __DO NOT SHARE IT OUTSIDE THE DATA OWNER PROCESS__.
 - __The Public key(or the encryption key)__. This key is used for encrypting the plain data to a ciphertext. You can safely share it with the Data Scientist.
 - __The Relinearization Keys(optional)__. This key is used for controlling the quality of the ciphertexts after encrypted multiplications. Generate it only if you are doing encrypted multiplications. You can safely share it with the Data Scientist.
 - __The Galois Keys(optional)__. This key is needed to perform encrypted vector rotation operations on ciphertexts. Generate it only if you are evaluating convolutions on encrypted data. You can safely share it with the Data Scientist.

### TenSEAL Context

Now that we had a short introduction, let's get to work.

The first step to do for a Data Owner is to generate a security context containing security parameters and encryption keys.

In [ ]:
context = ts.Context(
    ts.SCHEME_TYPE.CKKS,
    poly_modulus_degree=8192,
    coeff_mod_bit_sizes=[60, 40, 40, 60]
)
context.global_scale = 2**40
context

### Encrypt the data


In [ ]:
v1 = [0, 1, 2, 3, 4]
v2 = [4, 3, 2, 1, 0]

enc_v1 = ts.ckks_vector(context, v1)
enc_v2 = ts.ckks_vector(context, v2)
(enc_v1, enc_v2)

### Make Context and Encrypted Vectors Referenceable over Duet

In [ ]:
# tag them so our partner can easily reference it
ctx_ptr = context.send(duet, pointable=True, tags=["context"])
enc_v1_ptr = enc_v1.send(duet, pointable=True, tags=["enc_v1"])
enc_v2_ptr = enc_v2.send(duet, pointable=True, tags=["enc_v2"])

In [ ]:
# we can see that our three objects are now inside the store we control
duet.store.pandas

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 1 : Now STOP and run the Data Scientist notebook until the same checkpoint.

In [ ]:
# We can see our duet partner has requested the two encrypted vectors and the public context
duet.requests.pandas

### Approve the requests

In [ ]:
duet.requests[0].accept()
duet.requests[0].accept()
duet.requests[0].accept()

In [ ]:
# The requests should have been handled
duet.requests.pandas

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 2 : Now STOP and run the Data Scientist notebook until the same checkpoint.

### Get the computation results from store and decrypt them locally

In [ ]:
# Validate the encrypted add

result_add = duet.store["result_add"].get(delete_obj=False)
result_add.link_context(context)

result_add

In [ ]:
decrypted_result = result_add.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 + v2 for v1, v2 in zip(v1, v2)]

decrypted_result

In [ ]:
# Validate the encrypted - plain add

result_iadd = duet.store["result_iadd"].get(delete_obj=False)
result_iadd.link_context(context)

decrypted_result = result_iadd.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 + v2 for v1, v2 in zip(v1, [10, 10, 10, 10, 10])]

decrypted_result

In [ ]:
# Validate the encrypted subtraction

result_sub = duet.store["result_sub"].get(delete_obj=False)
result_sub.link_context(context)

decrypted_result = result_sub.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 - v2 for v1, v2 in zip(v1, v2)]

decrypted_result

In [ ]:
# Validate the encrypted multiplication

result_mul = duet.store["result_mul"].get(delete_obj=False)
result_mul.link_context(context)

decrypted_result = result_mul.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v1 * v2 for v1, v2 in zip(v1, v2)]

decrypted_result

In [ ]:
# Validate the encrypted power

result_pow = duet.store["result_pow"].get(delete_obj=False)
result_pow.link_context(context)

decrypted_result = result_pow.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [v ** 3 for v in v1]

decrypted_result

In [ ]:
# Validate the encrypted negation

result_neg = duet.store["result_neg"].get(delete_obj=False)
result_neg.link_context(context)

decrypted_result = result_neg.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [-v for v in v1]

decrypted_result

In [ ]:
# Validate the encrypted polynomial evaluation for 1 + X^2 + X^3

result_poly = duet.store["result_poly"].get(delete_obj=False)
result_poly.link_context(context)

decrypted_result = result_poly.decrypt()
assert pytest.approx(decrypted_result, abs=10**-3) == [1 + v**2 + v**3 for v in v1]

decrypted_result

### <img src="https://github.com/OpenMined/design-assets/raw/master/logos/OM/mark-primary-light.png" alt="he-black-box" width="100"/> Checkpoint 3 : Well done!

# Congratulations!!! - Time to Join the Community!

Congratulations on completing this notebook tutorial! If you enjoyed this and would like to join the movement toward privacy preserving, decentralized ownership of AI and the AI supply chain (data), you can do so in the following ways!

### Star PySyft and TenSEAL on GitHub

The easiest way to help our community is just by starring the Repos! This helps raise awareness of the cool tools we're building.

- [Star PySyft](https://github.com/OpenMined/PySyft)
- [Star TenSEAL](https://github.com/OpenMined/TenSEAL)

### Join our Slack!

The best way to keep up to date on the latest advancements is to join our community! You can do so by filling out the form at [http://slack.openmined.org](http://slack.openmined.org). #lib_tenseal and #code_tenseal are the main channels for the TenSEAL project.

### Donate

If you don't have time to contribute to our codebase, but would still like to lend support, you can also become a Backer on our Open Collective. All donations go toward our web hosting and other community expenses such as hackathons and meetups!

[OpenMined's Open Collective Page](https://opencollective.com/openmined)